In [3]:
import yaml
import add_packages
from pprint import pprint
import os, re
import pandas as pd
# import tqdm
from tqdm.auto import tqdm

from toolkit.langchain import (
	document_loaders, text_splitters, text_embedding_models, stores, 
	prompts, utils, output_parsers, agents, documents, models,
	runnables, tools, chains
)

from toolkit import sql

PATH_DATA = f"{add_packages.APP_PATH}/data/vtc"
FILE_CFG = "vtc.yaml"
tqdm.pandas(desc="Processing")

with open(f"{add_packages.APP_PATH}/my_configs/{FILE_CFG}", 'r') as file:
    configs = yaml.safe_load(file)

/opt/homebrew/Caskroom/miniconda/base/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
llm = models.chat_openai
my_sql_db = sql.MySQLDatabase()
embeddings = text_embedding_models.OpenAIEmbeddings()
vectorstore = stores.faiss.FAISS

# Data

## txt - FAQ

### vtc_faq

In [ ]:
path_txt = f"{PATH_DATA}/vtc_faq.txt"

In [ ]:
loader_txt = document_loaders.TextLoader(path_txt)
doc_txt = loader_txt.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
	# chunk_size=500, chunk_overlap=100,
	separators=["##"], chunk_size=150, chunk_overlap=0,
)
docs_txt = text_splitter.split_documents(doc_txt)
docs_txt = docs_txt[1:]

metadatas = {
	"data": "frequently asked questions"
}
utils.remove_metadata(docs_txt, "source")
utils.update_metadata(docs_txt, metadatas)

In [ ]:
docs_txt_vtc_faq = docs_txt

### onli_faq

In [ ]:
path_txt = f"{PATH_DATA}/onli_faq.txt"

In [ ]:
loader_txt = document_loaders.TextLoader(path_txt)
doc_txt = loader_txt.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
	# chunk_size=500, chunk_overlap=100,
	separators=["##"], chunk_size=150, chunk_overlap=0,
)
docs_txt = text_splitter.split_documents(doc_txt)
docs_txt = docs_txt[1:]

metadatas = {
	"data": "frequently asked questions"
}
utils.remove_metadata(docs_txt, "source")
utils.update_metadata(docs_txt, metadatas)

In [ ]:
docs_txt_onli_faq = docs_txt

### File 3

In [ ]:
path_txt = f"{PATH_DATA}/faq.txt"

In [ ]:
loader_txt = document_loaders.TextLoader(path_txt)
doc_txt = loader_txt.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
	# chunk_size=500, chunk_overlap=100,
	separators=["##"], chunk_size=150, chunk_overlap=0,
)
docs_txt = text_splitter.split_documents(doc_txt)
docs_txt = docs_txt[1:]

metadatas = {
	"data": "frequently asked questions"
}
utils.remove_metadata(docs_txt, "source")
utils.update_metadata(docs_txt, metadatas)

## table

### onli_course_list

In [5]:
file_csv = "onli_course_list.xlsx"
path_csv = f"{PATH_DATA}/{file_csv}"
path_csv_processed = f"{PATH_DATA}/{file_csv.split('.')[0]}2.xlsx"

In [6]:
df = pd.read_excel(
	path_csv, 
 	# delimiter="," # "," ";"
)

df.head()

,course_name,course_category,instructor_name,course_link,course_description
0,Hội Họa: Màu Sắc Và Phối Màu Cơ Bản,Thiết kế,Lê Hoàng Anh Thi,https://onlinica.com/courses/course-v1:Onlinic...,"Khóa học ""Màu sắc và cách phối màu cơ bản"" cun..."
1,Kỹ Năng Quản Lý Thời Gian,Phát triển cá nhân,Nguyễn Thu Hà,https://onlinica.com/courses/course-v1:Onlinic...,Khóa học này sẽ giúp bạn hiểu và áp dụng kỹ nă...
2,Kỹ Năng Sinh Tồn Trong Thành Phố,Phát triển cá nhân,Lê Diệu Hiền,https://onlinica.com/courses/course-v1:Onlinic...,Khóa học của Onlinica giúp trang bị kỹ năng tì...
3,Khám Phá Ngành Digital Marketing,Digital Marketing,Nguyễn Khánh Ly,https://onlinica.com/courses/course-v1:Onlinic...,"Khóa học ""Khám phá ngành Digital Marketing"" củ..."
4,Kỹ Năng Phỏng Vấn Tìm Việc Làm,Kỹ năng mềm,Bùi Thương Huyền,https://onlinica.com/courses/course-v1:Onlinic...,Khóa học này cung cấp các kỹ năng cần thiết để...


#### Process

In [18]:
model = models.chat_openai

template1 = """\
Here is a description of a learning course:

<course_description>
{{COURSE_DESCRIPTION}}
</course_description>

Please read the course description carefully. Your task is to extract the core information from the description, leaving out any boilerplate or filler words that are not essential to understanding the key points of the course.

Summarize the core information you have extracted in a single short paragraph. 

Provide your output in Vietnamese.
{text}"""

template2 = """\
...
{text}
"""

prompt_template1 = prompts.PromptTemplate.from_template(template1)
prompt_template2 = prompts.PromptTemplate.from_template(template2)

chain1 = prompt_template1 | model | output_parsers.StrOutputParser()
chain2 = prompt_template2 | model | output_parsers.StrOutputParser()

# chain = runnables.RunnablePassthrough.assign(
#   text=chain1
# ).assign(
#   text=chain2
# )

chain = runnables.RunnablePassthrough.assign(
  text=chain1
)

def process_csv_col(text: str) -> str:
  result = chain.invoke({"text": text})['text']
  return result

def capitalize_first_letter(s):
	return ' '.join([word.capitalize() for word in s.split()])

def change_col_value(df: pd.DataFrame, column_name: str, value, new_value):
	df[column_name] = df[column_name].replace(value, new_value)
	return df

def replace_col_value_if_contains(df, column_name, substring, new_substring):
	df[column_name] = df[column_name].str.replace(substring, new_substring)
	return df

# query = '...'
# result = process_csv_col(query)
# pprint(result)

In [ ]:
col_to_process = "course_name"

df[col_to_process] = df[col_to_process].progress_apply(process_csv_col)

df.to_excel(f"{path_csv_processed}", index=False)


In [ ]:
path_csv = path_csv_processed

#### Load

In [ ]:
path_csv

In [ ]:
csv_cols = list(df.columns)

loader_csv = document_loaders.UnstructuredExcelLoader(
	path_csv,
	mode="elements"
)
docs_csv = loader_csv.load_and_split()

In [ ]:
docs_csv = loader_csv.load()[1:]

metadatas = {
	"data": "..."
}

utils.remove_metadata(docs_csv, "source")
utils.remove_metadata(docs_csv, "row")
utils.update_metadata(docs_csv, metadatas)

#### Load to sql

In [10]:
my_table_schema = [
	"course_id SERIAL",
	"course_name VARCHAR(255) NOT NULL UNIQUE",
	"course_category VARCHAR(255) NOT NULL",
	"instructor_name VARCHAR(100) NOT NULL",
	"course_link VARCHAR(2048) NOT NULL UNIQUE",
	"course_description TEXT NOT NULL",
	"PRIMARY KEY (course_id)",
]
my_table = sql.MySQLTable(
	name="onli_course_list", 
	schema=my_table_schema,
	db=my_sql_db,
)
my_table.create()

db = stores.SQLDatabase.from_uri(my_sql_db.get_uri())
llm = models.chat_openai

embeddings = text_embedding_models.OpenAIEmbeddings()
vectorstore = stores.faiss.FAISS

table_cols = [col_description.split(" ")[0] for col_description in my_table_schema][1:-1]


In [12]:
my_table.schema

['course_id SERIAL',
 'course_name VARCHAR(255) NOT NULL UNIQUE',
 'course_category VARCHAR(255) NOT NULL',
 'instructor_name VARCHAR(100) NOT NULL',
 'course_link VARCHAR(2048) NOT NULL UNIQUE',
 'course_description TEXT NOT NULL',
 'PRIMARY KEY (course_id)']

In [ ]:
"Can you list all the courses that have 'Game Developer' in their name?",
"Which course has the shortest link?", "Are there any courses that have the same name but different instructors?"

In [ ]:
df = pd.read_excel(path_csv)
df.columns = table_cols

# my_table.insert_from_dataframe(df)

In [13]:
cols = ["course_name", "course_category", "instructor_name"]
proper_nouns_onli_course_list = [value for col in cols for value in my_table.get_discrete_values_col(col)]

In [ ]:
questions = [
  "Which courses are available in the Design category?",
	"Who are the instructors for the Personal Development courses?",
	"Can you provide a summary of the course descriptions for the Digital Marketing category?",
	"Which courses have the longest descriptions?",
	"How many courses are offered by each instructor?",
	"Which course categories have the most number of courses?",
	"Can you tell me about the course 'Kỹ năng quản lý thời gian'?",
	"Which courses have the shortest descriptions?",
	"Which instructors have the most number of courses?",
	"Can you list all the courses that are related to Personal Development?"
]

examples_questions_to_sql_onli_course_list = [
    {
        "input": "Which courses are available in the Design category?",
        "query": "SELECT course_name FROM courses WHERE course_category = 'Design';"
    },
    {
        "input": "Who are the instructors for the Personal Development courses?",
        "query": "SELECT DISTINCT instructor_name FROM courses WHERE course_category = 'Personal Development';"
    },
    {
        "input": "Can you provide a summary of the course descriptions for the Digital Marketing category?",
        "query": "SELECT course_name, SUBSTRING(course_description, 1, 100) AS summary FROM courses WHERE course_category = 'Digital Marketing';"
    },
    {
        "input": "Which courses have the longest descriptions?",
        "query": "SELECT course_name, LENGTH(course_description) AS description_length FROM courses ORDER BY description_length DESC LIMIT 5;"
    },
    {
        "input": "How many courses are offered by each instructor?",
        "query": "SELECT instructor_name, COUNT(course_id) AS num_courses FROM courses GROUP BY instructor_name;"
    },
    {
        "input": "Which course categories have the most number of courses?",
        "query": "SELECT course_category, COUNT(course_id) AS num_courses FROM courses GROUP BY course_category ORDER BY num_courses DESC LIMIT 5;"
    },
    {
        "input": "Can you tell me about the course 'Kỹ năng quản lý thời gian'?",
        "query": "SELECT * FROM courses WHERE course_name = 'Kỹ năng quản lý thời gian';"
    },
    {
        "input": "Which courses have the shortest descriptions?",
        "query": "SELECT course_name, LENGTH(course_description) AS description_length FROM courses ORDER BY description_length ASC LIMIT 5;"
    },
    {
        "input": "Which instructors have the most number of courses?",
        "query": "SELECT instructor_name, COUNT(course_id) AS num_courses FROM courses GROUP BY instructor_name ORDER BY num_courses DESC LIMIT 5;"
    },
    {
        "input": "Can you list all the courses that are related to Personal Development?",
        "query": "SELECT course_name FROM courses WHERE course_category = 'Personal Development';"
    }
]

### File vtc_course_list

In [17]:
file_xlsx = "vtc_course_list.xlsx"
path_xlsx = f"{PATH_DATA}/{file_xlsx}"
path_xlsx_processed = f"{PATH_DATA}/{file_xlsx.split('.')[0]}-prod.xlsx"

In [18]:
df = pd.read_excel(
	path_xlsx, 
 	# delimiter=";"
)

df.head()

,category,academic_program,program_type,reference_link,program_format,program_duration,program_time,no_enrollments,admission_targets,admission_form,program_schedule,program_introduction,program_outcomes,requirements,tuition,training_roadmap
0,Thiết kế,Thiết kế Dựng hình 3D (3D Modelling),Tiêu chuẩn,https://vtc.edu.vn/course/chuyen-vien-dung-hin...,Tập trung,Dài hạn,2 năm,390.0,- Các bạn trẻ yêu thiết kế và thích làm việc t...,- Đối tượng 1: Học sinh THPT\n - Đã tốt nghiệ...,- Giờ học: Ban ngày\n- Đợt 1: 04/2024\n- Đợt 2...,- Thiết kế 3D (3D Design) là công việc sử dụng...,- Được trang bị đầy đủ những kiến thức chuyên ...,"VTC Academy sẽ đào tạo từ đầu đến cuối, chương...",Liên hệ để được hỗ trợ chi tiết và nhận ưu đãi...,- Học kỳ 1: Nền tảng mỹ thuật\n - Định hướn...
1,Thiết kế,Hoạt hình/Diễn hoạt 3D ( 3D Animation),Tiêu chuẩn,https://vtc.edu.vn/course/chuyen-vien-dien-hoa...,Tập trung,Dài hạn,2 năm,390.0,- Các bạn trẻ yêu thiết kế và thích làm việc t...,- Đối tượng 1: Học sinh THPT\n - Đã tốt nghiệ...,- Giờ học: Ban ngày\n- Đợt 1: 04/2024\n- Đợt 2...,- Hoạt hình 3D (3D Animation) là công việc sử ...,- Được trang bị đầy đủ những kiến thức chuyên ...,"VTC Academy sẽ đào tạo từ đầu đến cuối, chương...",Liên hệ để được hỗ trợ chi tiết và nhận ưu đãi...,- Học kỳ 1: Nền tảng mỹ thuật\n - Định hướn...
2,Thiết kế,Thiết kế đồ họa (Digital Graphic Design),Tiêu chuẩn,https://vtc.edu.vn/course/chuyen-vien-thiet-ke...,Tập trung,Dài hạn,2 năm,390.0,- Những người muốn chuyển ngành sang thiết kế...,- Đối tượng 1: Học sinh THPT\n - Đã tốt nghiệ...,- Giờ học: Ban ngày\n- Đợt 1: 04/2024\n- Đợt 2...,- Thiết kế đồ họa là công việc sử dụng các côn...,- Được trang bị đầy đủ những kiến thức chuyên ...,"VTC Academy sẽ đào tạo từ đầu đến cuối, chương...",Liên hệ để được hỗ trợ chi tiết và nhận ưu đãi...,- Học kỳ 1: Nền tảng mỹ thuật\n - Định hướn...
3,Công nghệ thông tin,Lập trình Game (Game Development),Tiêu chuẩn,https://vtc.edu.vn/course/chuyen-vien-phat-tri...,Tập trung,Dài hạn,2 năm,390.0,- Những người muốn trở thành Chuyên viên Lập t...,- Đối tượng 1: Học sinh THPT\n - Đã tốt nghiệ...,- Giờ học: Ban ngày\n- Đợt 1: 04/2024\n- Đợt 2...,- Lập trình game là một phần quan trọng của ng...,- Được trang bị đầy đủ những hiểu biết về quá ...,"VTC Academy sẽ đào tạo từ đầu đến cuối, chương...",Liên hệ để được hỗ trợ chi tiết và nhận ưu đãi...,- Học kỳ 1: Cơ sở lập trình\n - Định hướng ...
4,Công nghệ thông tin,Lập trình Phần mềm (Full-stack),Tiêu chuẩn,https://vtc.edu.vn/course/chuyen-vien-phat-tri...,Tập trung,Dài hạn,2 năm,390.0,- Những ai có đam mê và mong muốn trở thành Ch...,- Đối tượng 1: Học sinh THPT\n - Đã tốt nghiệ...,- Giờ học: Ban ngày\n- Đợt 1: 04/2024\n- Đợt 2...,- Lập trình phần mềm là một trong những lĩnh v...,- Được trang bị đầy đủ kiến thức về quá trình ...,"VTC Academy sẽ đào tạo từ đầu đến cuối, chương...",Liên hệ để được hỗ trợ chi tiết và nhận ưu đãi...,- Học kỳ 1: Cơ sở lập trình\n - Định hướng ...


In [19]:
df["academic_program"].unique()

array(['Thiết kế Dựng hình 3D (3D Modelling)',
       'Hoạt hình/Diễn hoạt 3D ( 3D Animation)',
       'Thiết kế đồ họa (Digital Graphic Design)',
       'Lập trình Game (Game Development)',
       'Lập trình Phần mềm (Full-stack)',
       'Digital Marketing (Full-stack)',
       'Quản lý chuỗi cung ứng và Logistics', 'Kỹ xảo Hoạt hình 3D',
       'Thiết kế 3D nâng cao', 'Lập trình Phần mềm', 'Trí tuệ nhân tạo',
       'Họa Sỹ Nhân Vật 3D Chuyên Nghiệp (3D Character Artist Pro)',
       'Họa Sỹ Môi Trường 3D Chuyên Nghiệp (3D Environment Artist Pro)',
       'AI For Everyone', 'Python Developer For AI',
       'AI Specialist - AI Engineer', 'AI Specialist - Data Scientist',
       'AI Expert - AI Engineer', 'AI Expert - Data Scientist',
       'Data Analytics For BI/BA', 'AI Practitioner',
       'Visual Understanding',
       'Level 2 - AI Specialist - Language Understanding',
       'Level 2 - AI Specialist - Decision Making',
       'AI Full-stack Engineer', 'Tools Master For Digita

#### Process

In [ ]:
model = models.chat_openai

template1 = """\
...
{text}"""

template2 = """\
...
{text}
"""

prompt_template1 = prompts.PromptTemplate.from_template(template1)
prompt_template2 = prompts.PromptTemplate.from_template(template2)

chain1 = prompt_template1 | model | output_parsers.StrOutputParser()
chain2 = prompt_template2 | model | output_parsers.StrOutputParser()

chain = runnables.RunnablePassthrough.assign(
  text=chain1
).assign(
  text=chain2
)

def process_xlsx_col(text: str) -> str:
  result = chain.invoke({"text": text})['text']
  return result

query = '...'
result = process_xlsx_col(query)

pprint(result)

In [ ]:
df = pd.read_excel(path_xlsx)

col_to_process = "..."

df[col_to_process] = df[col_to_process].progress_apply(process_xlsx_col)

df.to_excel(f"{path_xlsx_processed}", index=False)


In [ ]:
path_xlsx = path_xlsx_processed

#### Load

In [ ]:
xlsx_cols = ...

loader_xlsx = document_loaders.UnstructuredExcelLoader(
	path_xlsx,
	mode="elements",
)
docs_xlsx = loader_xlsx.load()

metadatas = {
	"data": "..."
}

utils.remove_metadata(docs_xlsx, "source")
utils.remove_metadata(docs_xlsx, "row")
utils.update_metadata(docs_xlsx, metadatas)

#### Load to sql

In [ ]:
# Khối ngành	Chuyên ngành	Chương trình	Link tham khảo	Hình thức học	Độ dài	Thời gian	Chỉ tiêu học viên	Đối tượng tuyển sinh	Hình thức tuyển sinh	Lịch khai giảng	Giới thiệu khoá học	Đầu ra khoá học	Yêu cầu	Học phí

In [20]:
my_table_schema = [
	"id SERIAL",
	"category VARCHAR(255) NOT NULL",
	"academic_program VARCHAR(255) NOT NULL",
	"program_type VARCHAR(255) NOT NULL",
	"reference_link VARCHAR(2048) NOT NULL",
	"program_format VARCHAR(255) NOT NULL",
	"program_duration VARCHAR(255) NOT NULL",
	"program_time VARCHAR(128) NOT NULL",
	"no_enrollments TEXT",
	"admission_targets TEXT",
	"admission_form TEXT",
	"program_schedule TEXT",
	"program_introduction TEXT",
	"program_outcomes TEXT",
	"requirements TEXT",
	"tuition TEXT",
	"PRIMARY KEY (id)",
	"training_roadmap TEXT"
]

my_table = sql.MySQLTable(
	name="vtc_course_list", 
	schema=my_table_schema,
	db=my_sql_db,
)
my_table.create()

db = stores.SQLDatabase.from_uri(my_sql_db.get_uri())
llm = models.chat_openai

embeddings = text_embedding_models.OpenAIEmbeddings()
vectorstore = stores.faiss.FAISS

table_cols = [col_description.split(" ")[0] for col_description in my_table_schema][1:-1]


In [ ]:
# df = pd.read_excel(path_xlsx)
# df.columns = table_cols

In [21]:
my_table.insert_from_dataframe(df)

In [ ]:
cols = [
  "category", "academic_program", "program_type", "program_format", 
  "program_duration", "program_time"
]
proper_nouns_vtc_course_list = [value for col in cols for value in my_table.get_discrete_values_col(col)]

### FINAL SQL TABLE

In [ ]:
examples_questions_to_sql = []
proper_nouns = []

proper_nouns.extend(value for value in proper_nouns_onli_course_list)
proper_nouns.extend(value for value in proper_nouns_vtc_course_list)

# Vector store 

Note:
- `tiktoken` >= 0.6.0

## txt

### vtc_faq

In [ ]:
qdrant_txt_vtc_faq = stores.QdrantWrapper(
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  configs=configs,
  **configs["vector_db"]["qdrant"]["vtc_faq"]
)

In [ ]:
# qdrant_txt_vtc_faq.add_documents(docs_txt_vtc_faq)

### onli_faq

In [ ]:
qdrant_txt_onli_faq = stores.QdrantWrapper(
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  configs=configs,
  **configs["vector_db"]["qdrant"]["onli_faq"]
)

In [ ]:
# qdrant_txt_onli_faq.add_documents(docs_txt_onli_faq)

### File 3

In [ ]:
qdrant_txt = stores.QdrantWrapper(
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  configs=configs,
  **configs["vector_db"]["qdrant"]["..."]
)

In [ ]:
# qdrant_txt.add_documents(docs_txt)

## table

### File 1

In [ ]:
qdrant_csv = stores.QdrantWrapper(
	qdrant_host=os.getenv("QDRANT_HOST"),
	qdrant_api_key=os.getenv("QDRANT_API_KEY"),
	configs=configs,
	**configs["vector_db"]["qdrant"]["..."]
)

In [ ]:
qdrant_csv.add_documents(docs_csv)

### File 2

In [ ]:
qdrant_csv = stores.QdrantWrapper(
	qdrant_host=os.getenv("QDRANT_HOST"),
	qdrant_api_key=os.getenv("QDRANT_API_KEY"),
	configs=configs,
	**configs["vector_db"]["qdrant"]["..."]
)

In [ ]:
qdrant_csv.add_documents(docs_csv)

# Test

In [ ]:
# qdrant_lectures_content = stores.QdrantWrapper(
#   qdrant_host=os.getenv("QDRANT_HOST"),
#   qdrant_api_key=os.getenv("QDRANT_API_KEY"),
#   configs=configs,
#   **configs["vector_db"]["qdrant"]["lectures_content"],
# )

In [ ]:
examples_fewshot_tmp = dict(configs["sql"]["examples_questions_to_sql"]).values()
examples_questions_to_sql = [example for sublist in examples_fewshot_tmp for example in sublist]

proper_nouns = configs["sql"]["proper_nouns"]

my_sql_db = sql.MySQLDatabase()

cfg_sql_tool = configs["sql"]["tool"]

my_sql_chain = chains.MySqlChain(
	my_sql_db=my_sql_db,
	llm=llm,
	embeddings=embeddings,
	vectorstore=vectorstore,
	proper_nouns=proper_nouns,
	k_retriever_proper_nouns=4,
	examples_questions_to_sql=examples_questions_to_sql,
	k_few_shot_examples=5,
	# is_debug=True,
	tool_name=cfg_sql_tool["name"],
	tool_description=cfg_sql_tool["description"],
	tool_metadata=cfg_sql_tool["metadata"],
	tool_tags=cfg_sql_tool["tags"],
)

tool_chain_sql = my_sql_chain.create_tool_chain_sql()

In [ ]:
llm = models.chat_openai

tools = [
	qdrant_txt_vtc_faq.retriever_tool,
	qdrant_txt_onli_faq.retriever_tool,
	tool_chain_sql,
]

system_message_custom = configs["prompts"]["system_message_vtc"]
prompt = prompts.create_prompt_tool_calling_agent(system_message_custom)

agent = agents.MyStatelessAgent(
	llm=llm,
	tools=tools,
	prompt=prompt,
	agent_type=configs["agents"]["type"],
	agent_verbose=False,
)

In [ ]:
questions = [
  # VTCA FAQs
	"Các đặc quyền của học viên VTC Academy",
	"Thành tựu của VTC Academy",
	"Sứ mệnh của VTC Academy",
	"Tầm nhìn của VTC Academy",
	"Mục tiêu của VTC Academy",
	"Thông tin liên hệ các cơ sở của VTC Academy",
	"Mô hình đào tạo tại VTC Academy",
	"Không gian tại VTC Academy",
	"Đội ngũ giảng viên tại VTC Academy",
	"Đối tượng tuyển sinh tại VTC Academy",
	"Hình thức tuyển sinh tại VTC Academy",
	"Quy trình đăng ký và làm hồ sơ nhập học tại VTC Academy",
	"Hồ sơ nhập học trực tuyến tại VTC Academy",
	"Lịch khai giảng các khoá học tại VTC Academy",
	"Tôi muốn nộp hồ sơ nhập học online cho con tôi vì đang ở một thành phố khác, làm sao có thể làm thủ tục nhập học tại VTC Academy?",
	"Hồ sơ nhập học trực tuyến (online) tại VTC Academy cần có những gì?",
	"Làm sao để biết con tôi đã được nhập học thành công tại VTC Academy?",
	"Lịch khai giảng của Nhà trường tại VTC Academy là bao giờ?",
	"Nếu học online thì chất lượng và lộ trình học tập có đảm bảo không? Hình thức học online tại VTC Academy như thế nào?",
	"Học viên có cần phải thi tuyển đầu vào tại VTC Academy không?",
	"Tôi muốn tham quan cơ sở nhưng vì ở xa nên không có điều kiện đến tận trường. Có thể chia sẻ hình ảnh môi trường học tập thực tế tại các cơ sở ",
	"VTC Academy có các hình thức hoặc chương trình ưu đãi nào để hỗ trợ gia đình học viên không?",
	"Các hình thức đóng học phí cho nhà trường tại VTC Academy?",
	"Tôi muốn đóng học phí trực tiếp cho trường VTC Aacademy vì không đủ điều kiện để đóng qua Internet Banking hay qua thẻ thì nhà trường có hỗ ",
	"Nhà trường tại VTC Academy có hỗ trợ tìm nhà trọ không?",
	"Sau khi hoàn thành các khoá học tại VTC Academy, học viên có việc làm ngay không?",
	"Đăng ký xét học bạ tại VTC Academy",
	"VTC Academy Plus tại VTC Academy",
	"Học bổng tài năng tại VTC Academy",
	"Học bổng tài năng tại VTC Academy Plus",
	"Thông tin tham khảo về tuyển sinh ở VTC Academy",
	"Hình thức xét học bạ tại VTC Academy như thế nào?",
	"Điều kiện đăng ký xét tuyển các ngành học tại VTC Academy là gì?",
	"Tôi không có bằng THPT, liệu có thể học tại VTC Academy được không?",
	"VTC Academy có hỗ trợ nơi ở cho những bạn ở xa theo học tại trường không?",
	"Để nhập học tại VTC Academy cần thi khối nào, môn thi là gì, điểm chuẩn là bao nhiêu?",
	"Sau khi xét học bạ tại VTC Academy thì có cần đăng ký trên cổng thông tin bộ giáo dục không?",
	"Bằng tốt nghiệp tại VTC Academy thuộc hệ gì?",
	"Bằng tốt nghiệp tại VTC Academy có xin được bên ngoài không hay chỉ doanh nghiệp liên kết với trường?",
	"Khi nào thì nhận được chứng chỉ DMI tại VTC Academy?",
	"Bằng khóa Plus tại VTC Academy khác gì so với khóa thường?",
	"Trường tại VTC Academy có các cơ sở nào?",
	"Giấy tờ đăng ký và nhập học tại VTC Academy cần những gì?",
	"Thời gian học tại VTC Academy có phù hợp cho sinh viên học 2 trường/người đi làm không?",
	"Sau khi tốt nghiệp, tôi có thể làm việc ở những công ty nào mà VTC Academy liên kết?",
	"Học phí giữa các cơ sở tại VTC Academy có chênh lệch không?",
	"Thời gian học 1 khóa học tại VTC Academy khoảng mấy năm?",
	"Thời gian nhập học tại VTC Academy là khi nào?",
	"Hạn chót gửi hồ sơ xét tuyển tại VTC Academy là khi nào?",
	"Tôi có thể dời thời gian nhập học tại VTC Academy không? Nếu có thì được dời lại trong bao lâu?",
	"Khi đăng ký nhập học tại VTC Academy có được miễn nghĩa vụ quân sự không?",
	"Trường tại VTC Academy có lớp buổi tối không?",
	"Em không biết chọn ngành nào tại VTC Academy?/Em không biết mình phù hợp với ngành nào?",
	"Tôi chưa đủ tuổi thì có được theo học tại VTC Academy không?",
	"VTC Academy có khóa học online không?",
	"Tôi học song song tại VTC Academy với trường trung học có được không? Yêu cầu gì?",
	"VTC Academy có cam kết việc làm bằng văn bản pháp lý thật không?",
	"Học tại VTC Academy có liên thông được không?",
	"Trong thời gian học hệ Plus tại VTC Academy có đi học ở các trường liên kết nước ngoài không?",
	"Trường có tăng học phí trong thời gian học tại VTC Academy không?",
	"Trường đang có các ưu đãi gì khi đăng ký nhập học tại VTC Academy không?",
	"Thông tin chi tiết về trả góp học phí 0% tại VTC Academy ",
	"Mê game thì có thể học những ngành học nào tại VTC Academy?",
	"Có chương trình hỗ trợ học phí tại VTC Academy hay ưu đãi gì không?",
	"Có ưu đãi gì cho Quân Nhân Xuất Ngũ tại VTC Academy không?",
 
	# VTCA courses
  "Các khối ngành đào tạo tại VTCA?",
	"Khối ngành thiết kế tại VTC Academy có những chuyên ngành nào?",
	"Thời lượng và hình thức học của khóa học lập trình phần mềm tại VTCA",
	"Ở VTCA có những khoá học dài hạn nào?",
	"Ở VTCA có những khoá học trung hạn nào?",
	"Đối tượng nhập học của khóa học thiết kế 3D tại VTC Academy",
	"Các khoá học tại VTCA có hình thức học tập trung",
	"Link tham khảo khóa học quản lý chuỗi ứng dụng và logistic tại VTC Academy",
	"Học phí cho khóa học Digital Marketing VTCA",
	"Thời gian khai giảng khóa học 3D Modeling tại VTCA",
	"Cơ hội nghề nghiệp của ngành học digital marketing ở VTCA?",
	"Các chương trình học tiêu chuẩn tại VTCA",
	"Các chương trình học plus của VTCA",
  
  # Onlinica courses
	"Các thể loại khoá học ở Onlinica", # redo onli_faq
	"Giảng viên nào có nhiều khoá học nhất ở Onlinica",
	"Các Giảng viên có nhiều khoá học nhất ở Onlinica",
	"Thể loại khoá học nào có nhiều khoá học nhất ở Onlinica?",
	"Các khoá học của thầy Trần Anh Tuấn ở Onlinica", 
	"Các khoá học về lập trình ở Onlninica",
	"Các giảng viên dạy về lập trình ở Onlninica",
	"Thông tin khoá học kỹ năng quản lý thời gian ở Onlinica?",
	"Giảng viên của khoá học 'Kỹ Năng Giao Tiếp Hiệu Quả' ở Onlinica",
	"Các khoá học về game ở Onlinica", #
 
	# Onlinica FAQs
	"Làm cách nào để đăng ký tài khoản Onlinica?", 
	"Có thể thanh toán Onlinica bằng Momo được không",
	"Có mấy loại tài khoản Onlinica?",
	"Các khoá học ở Onlinica có mất phí không?",
	"Các khoá học của tôi tại Onlinica có thời hạn sử dụng bao lâu?",
	"Tôi có thể xoá tài khoản Onlinica không? Và làm như thế nào?",
	"Tôi có bị mất phí khi hết thời gian trải nghiệm Onlinica VIP không?",
	"Tôi sẽ nhận được chứng chỉ hoàn thành khóa học chứ? Và sẽ nhận qua hình thức nào?",
	"Vì sao tôi chưa nhận được chứng chỉ hoàn thành khi đã học xong?",
	"Onlinica có mấy hình thức thanh toán?",
	"Làm cách nào để đăng ký tài khoản Onlinica?",
	"Có mấy loại tài khoản Onlinica?",
	"Các khoá học ở Onlinica có mất phí không?",
	"Các khoá học của tôi tại Onlinica có thời hạn sử dụng bao lâu?",
	"Tôi có thể xoá tài khoản Onlinica không? Và làm như thế nào?",
	"Tôi có bị mất phí khi hết thời gian trải nghiệm Onlinica VIP không?",
	"Onlinica hiện đang có các khóa học nào?",
	"Tôi sẽ nhận được chứng chỉ hoàn thành khóa học chứ? Và sẽ nhận qua hình thức nào?",
	"Vì sao tôi chưa nhận được chứng chỉ hoàn thành khi đã học xong?",
	"Onlinica có mấy hình thức thanh toán?",
]

In [ ]:
res = []
async for chunk in agent.astream_events_basic(
  "Khối ngành thiết kế tại VTC Academy có những chuyên ngành nào?",
  history_type="in_memory",
):
	print(chunk, end="", flush=True)
	res.append(chunk)

In [ ]:
result = await agent.invoke_agent(
  "Khối ngành thiết kế tại VTC Academy có những chuyên ngành nào?",
	history_type="in_memory"
)

pprint(result)

# Todos

- [ ] Deep dive into chain

['id SERIAL',
 'major VARCHAR(255) NOT NULL',
 'academic_program VARCHAR(255) NOT NULL',
 'program_type VARCHAR(255) NOT NULL',
 'reference_link VARCHAR(2048) NOT NULL',
 'program_format VARCHAR(255) NOT NULL',
 'program_duration VARCHAR(255) NOT NULL',
 'program_time VARCHAR(128) NOT NULL',
 'no_enrollments TEXT',
 'admission_targets TEXT',
 'admission_form TEXT',
 'program_schedule TEXT',
 'program_introduction TEXT',
 'program_outcomes TEXT',
 'requirements TEXT',
 'tuition TEXT',
 'PRIMARY KEY (id)']

